In [ ]:
%load_ext autoreload
%autoreload 2

import bnl
import features as ft

slm_ds = bnl.Dataset("~/data/salami/metadata.csv")

# Get a track, and it's features


In [ ]:
track = slm_ds[626]
audio_path = track.info['audio_mp3_path']
audio_path

In [ ]:
f, ts = ft.yamnet_emb(audio_path)
ft.plot(f, ts)

In [ ]:
import numpy as np
import features as ft

feat_path = str(audio_path).replace("/audio.mp3", "_feats.npz").replace("audio", "features")
feats = np.load(feat_path)

ft.plot(feats, feat_type='crema')

# follow ssdm and get the SSM + path_sim

In [ ]:
feats['crema'].shape

In [ ]:
import librosa
import numpy as np
from scipy import sparse

def mask_diag(sq_mat, width=2):
    # carve out width from the full ssm
    sq_mat_lil = sparse.lil_matrix(sq_mat)
    for diag in range(-width + 1, width):
        sq_mat_lil.setdiag(0, diag)
    return sq_mat_lil.toarray()

In [ ]:
import matplotlib.pyplot as plt
rep_f_tyep = 'yamnet'

# I need to make sure that the features is longer than width * 2 + 1!
data_len = len(feats['ts']) - 1
max_width = ((data_len - 1) // 2) - 1
width = min(5, max_width)

# add a tiny padding to prevent zero division in cosine distance
A_rep = librosa.feature.stack_memory(feats[rep_f_tyep], n_steps=4, delay=1)

R = librosa.segment.recurrence_matrix(
    A_rep, metric='sqeuclidean', mode='affinity', bandwidth='med_k_scalar', sym=True, full=False, width=width
)

plt.imshow(R)
plt.colorbar();

In [ ]:

plt.imshow(R_sparse)
plt.show()

In [ ]:
import scipy
def embed_features(A_rep, A_loc, 
                   config={'rec_width': 13,
                           'rec_smooth': 7,
                           'evec_smooth': 13,
                           'rep_metric': 'cosine'}):

    R = librosa.segment.recurrence_matrix(A_rep, width=config['rec_width'],
                                          mode='affinity',
                                          metric=config['rep_metric'],
                                          sym=True)
    
    
    # Enhance diagonals with a median filter (Equation 2)
    df = librosa.segment.timelag_filter(scipy.ndimage.median_filter)
    Rf = df(R, size=(1, config['rec_smooth']))

    path_distance = np.sum(np.diff(A_loc, axis=1)**2, axis=0)
    sigma = np.median(path_distance)
    path_sim = np.exp(-path_distance / sigma)

    R_path = np.diag(path_sim, k=1) + np.diag(path_sim, k=-1)

    # And compute the balanced combination (Equations 6, 7, 9)
    deg_path = np.sum(R_path, axis=1)
    deg_rec = np.sum(Rf, axis=1)

    mu = deg_path.dot(deg_path + deg_rec) / np.sum((deg_path + deg_rec)**2)

    A = mu * Rf + (1 - mu) * R_path
    return A

    

In [ ]:

# follow ssdm and get the combined_graph


In [ ]:

# Get the proper laplacian


In [ ]:
#####################################################
# Now let's compute the normalized Laplacian (Eq. 10)
L = scipy.sparse.csgraph.laplacian(A, normed=True)

# and its spectral decomposition
evals, evecs = scipy.linalg.eigh(L)

# We can clean this up further with a median filter.
# This can help smooth over small discontinuities
evecs = scipy.ndimage.median_filter(evecs, size=(config["evec_smooth"], 1))

return evecs

In [ ]:

# Get the evecs and get to a bnl.MS object as output